In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from simclr import get_dataloader
from simclr_model import SimCLR_ResNet18
from simclr import SimCLR
import torch

n_views = 5
batch_size = 512
feature_dims = 128
lr = 0.0003
weight_decay = 1e-4
temperature = 0.07
log_dir = "./cifar_logs"

cifar10_loader = get_dataloader(
    "./data", "CIFAR10", n_views=n_views, batch_size=batch_size
)


# stl10 = DataSet(data_folder)
# stl10.data_aug(size=96, kernel_size=(5, 5), n_views=5)
# stl10.get_dataset("stl10")

Files already downloaded and verified


In [5]:
model = SimCLR_ResNet18(feature_dims)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=len(cifar10_loader), eta_min=0, last_epoch=-1
)
simclr = SimCLR(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_size=batch_size,
    n_views=n_views,
    temperature=temperature,
    log_dir=log_dir,
    checkpoint_dir="./cifar10",
)
simclr.train(cifar10_loader, 100, log_every_n_steps=5)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [3:33:28<00:00, 128.08s/it]


In [9]:
import torch

torch.save(model.state_dict(), "./cifar10/final_cifar10.pth")

stl10_loader = get_dataloader("./data", "stl10", n_views=n_views, batch_size=batch_size)
log_dir = "./stl_logs"
model = SimCLR_ResNet18(feature_dims)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=len(stl10_loader), eta_min=0, last_epoch=-1
)
simclr = SimCLR(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_size=batch_size,
    n_views=n_views,
    temperature=temperature,
    log_dir=log_dir,
    checkpoint_dir="./stl10",
)
simclr.train(stl10_loader, 100, log_every_n_steps=5)
torch.save(model.state_dict(), "./stl10/final_stl10.pth")

Files already downloaded and verified


 58%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                        | 58/100 [10:30:05<7:46:04, 665.81s/it]